<span style="font-size:12px;">

| 단계 | 데이터 형태 | 차원 예시 (Shape) | 핵심 역할 및 가능 여부 |
| :---: | :---: | :--- | :--- |
| **1. 원시 입력** | **문자열 (String)** | N/A | **[가능]** 사람이 읽고, 전처리(공백/Prefix 추가) 가능.<br>**[불가능]** 모델의 연산, 학습에 직접 사용 불가능. |
| **2. 토큰 ID** | **정수 텐서 (Integer Tensor)** | `(Batch Size, Seq Len)` | **[필수]** 모델에 입력되는 최초의 숫자 형태. 임베딩 테이블의 **인덱스** 역할. |
| **3. 임베딩** | **실수 텐서 (Float Tensor)** | `(Batch Size, Seq Len, Hidden Size)` | **[필수]** **벡터(1차원 텐서)**의 모음. 문맥 정보를 담으며, 모든 수학 연산 및 학습이 가능한 형태로 변환됨. |
| **4. 모델 연산** | **실수 텐서 (Float Tensor)** | `(Batch Size, Seq Len, Hidden Size)` | **[가능]** 어텐션 및 피드포워드 과정을 거치며 문맥 이해도를 높임. (계속해서 형태 유지) |
| **5. 최종 출력** | **로짓 (Logits)** | `(Batch Size*Seq Len, Num Labels)` | **[필수]** **선형 레이어(분류 헤드)**를 통해 라벨의 개수($N_{labels}$)에 맞춰 차원이 투영됨. 손실 계산의 기준점. |
| **6. 최종 결과** | **문자열 (String)** | N/A | **[필수]** **디코더**를 통해 로짓에서 나온 예측 ID를 다시 사람이 읽을 수 있는 텍스트로 변환하여 출력. |

<span style="color: lightblue;"> HuggingFace Transformers  

from transformers import (  
    pipeline, ······················ # 고수준 API - 가장 쉬운 방법  
    AutoTokenizer, ················· # 자동 토크나이저  
    AutoModelForSeq2SeqLM, ········· # 자동 모델 로더 (Seq2Seq 작업용)  
    T5TokenizerFast, ··············· # T5 전용 고속 토크나이저  
    T5ForConditionalGeneration, ···· # T5 모델 클래스  
    BartForConditionalGeneration, ·· # BART 모델 클래스  
    PreTrainedTokenizerFast, ······· # 사전학습 토크나이저 기본 클래스  
    DataCollatorForSeq2Seq, ········ # Seq2Seq 학습용 데이터 콜레이터  
    Seq2SeqTrainingArguments, ······ # Seq2Seq 학습 하이퍼파라미터  
    Seq2SeqTrainer, ················ # Seq2Seq 전용 트레이너  
)


In [8]:
# 경고메시지 숨기기
import warnings
warnings.filterwarnings("ignore")
import os
import torch
import numpy as np
from datetime import datetime

In [9]:
# 데이터셋 라이브럴
from datasets import load_dataset

In [10]:
# 버전확인
print(f'pytorch 버전 : {torch.__version__}')
print(f"transformer 버전 : {__import__('transformers').__version__}")

pytorch 버전 : 2.9.1+cpu
transformer 버전 : 4.57.1


---

<span style="color: Gold"> pipeline을 이용한 간단한 문서 요약

<span style="font-size:12px;">

- 장점: 적은 코드로 실행가능(2~3줄)
- 단점: 세밀한 제어 어려움
- 활용:
    - 빠른 프로토타입 제작
    - 간단한 데모
    - 성능테스트  
<br>

- 과정
1. 모델 다운로드 및 로딩
2. 토크나이제이션(문자-> 숫자)
3. 모델추론(요약생성)
4. 디코딩(숫자-> 문자)  

<br>

- `pipeline에 넣어야 하는 것`
    - Task 이름 (예:task="summarization/sentiment-analysis)
    - 모델 이름
    - 토크나이저(생략가능. 모델이름은 넣으면 그에 맞는 토크나이저가 자동으로 불러와진다)
    - 디바이스 옵션 생략 가능  
=> <span style="color: yellow">파이프라인은 항상 리스트 형태로 결과를 반환하기 때문에 뒤에 인덱스를 써야 요약문을 꺼낼 수 있음


In [11]:

from transformers import pipeline
summarizer = pipeline (
    'summarization', #작업유형
    model = "facebook/bart-large-cnn", #facebook 에서 개발한 요약모델(생성형)
    device = 0 if torch.cuda.is_available() else -1 

)

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

summary_result = summarizer(ARTICLE,
                            max_length = 130,
                            min_length = 30,
                            do_sample = False)  # 매번 동일한 결과(재현성)
print(summary_result[0]['summary_text'])

Device set to use cpu


Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.


In [12]:
summary_text = summary_result[0]['summary_text']
# 통계분석
compression_ratio = len(summary_text)/len(ARTICLE)*100
word_reduced = len(ARTICLE.split()) - len(summary_text.split())
print(f'원문 길이 : {len(ARTICLE):4d}문자({len(ARTICLE.split()):3d}단어)')
print(f'요약문 길이 : {len(summary_text):4d}문자({len(summary_text.split()):3d}단어)')
print(f'압축률 : {compression_ratio:.1f}%')
print(f'단어감소 : {word_reduced} 단어절감')



원문 길이 : 2298문자(369단어)
요약문 길이 :  262문자( 47단어)
압축률 : 11.4%
단어감소 : 322 단어절감


---

<span style="color: Gold"> T5 모델 활용
- google에서 2019년 출시
- 모든 NLP 작업을 텍스트 → 텍스트 형식으로 통일
- 입력 → 출력 모두 텍스트로 처리 → 범용 NLP 모델
- 사용 예시(<span style="color: yellow">Task Perfix</span> 필요)
    - `요약` : 'summarize : [원문] -> [요약문]'
    - `번역` : 'translate English to French : [원문] -> [번역문]'
    - `분류` : 'sentiment [원문] -> [클래스]'    l love this! -> positive  
=> 단점 : Task Prefix 필수 (없으면 급격한 성능 저하)

In [ ]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
# gpu 이동(가능하면)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

# 전처리
# 1. 공백제거
# 2. 줄바꿈을 공백으로 변환 -> 모델은 줄바꿈을 잘 인식 못함
# 3. Task Prefix 추가 ** 필수**
preprecess_text = ARTICLE.strip().replace('\n'," ")
input_text = f"summarize: {preprecess_text}"

# 토크나이제이션(인코딩)
tokenized_text = tokenizer.encode(
    input_text,return_tensors = 'pt',
    truncation = True, max_length = 512).to(device)

print(f'토큰수: {tokenized_text.shape[1]}')
print(f'텐서형태: {tokenized_text.shape}(배치크기 =1, 시퀀스길이={tokenized_text.shape[1]})')
print(f'첫10개토큰: {tokenized_text[0][:10].tolist()}')

# 디코딩(숫자 -> 텍스트)
decoded_sample = tokenizer.decode(
    tokenized_text[0][:20],  # 처음 20개 토큰
    skip_special_tokens=True  # 특수 토큰 제거
)

print(f'첫 20개 토큰 디코딩: {decoded_sample}')


토큰수: 512
텐서형태: torch.Size([1, 512])(배치크기 =1, 시퀀스길이=512)
첫10개토큰: [21603, 10, 368, 1060, 41, 254, 17235, 61, 10555, 301]
첫 20개 토큰 디코딩: summarize: New York (CNN)When Liana Barrientos was 23 years old,


---

<span style="color: Gold"> Beam Search</span> : 여러 가능성을 동시에 탐색하면서 최적의 요약 찾기

In [ ]:
# beam = 4 ,min_length=30, max_length =100

summary_ids = model.generate(
    tokenized_text,
    num_beams = 4,
    no_repeat_ngram_size = 3, # 3-gram 반복 방지
    min_length=30,
    max_length =100,
    early_stopping = True # EOS 만나면 종료
)
# 생성완료
# 디코딩
output = tokenizer.decode(summary_ids[0],skip_special_tokens = True)
print(f'요약 통계')
print(f'원문의 길이: {len(ARTICLE):4d} 문자')
print(f'요약문의 길이: {len(output):4d} 문자')
print(f'압축률: {len(output) / len(ARTICLE) *100:.1f}%')
print(f'생성 토큰수 : {summary_ids.shape[1]}개')
print(f'요약문: {output} ')


요약 통계
원문의 길이: 2298 문자
요약문의 길이:  217 문자
압축률: 9.4%
생성 토큰수 : 54개
요약문: in 2010, she married once more, this time in the Bronx. she is facing two criminal counts of "offering a false instrument for filing in the first degree" prosecutors say the marriages were part of an immigration scam. 


---

실습  
<span style="color: pink"> 1. 영어 -> 독일어로 바꾸기

In [27]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
# gpu 이동(가능하면)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

# 전처리
# 1. 공백제거
# 2. 줄바꿈을 공백으로 변환 -> 모델은 줄바꿈을 잘 인식 못함
# 3. Task Prefix 추가 ** 필수**
preprecess_text = ARTICLE.strip().replace('\n'," ")
input_text = f"translate English to German: {preprecess_text}"

# 토크나이제이션(인코딩)
tokenized_text = tokenizer.encode(
    input_text,return_tensors = 'pt',
    truncation = True, max_length = 512).to(device)

# 디코딩(숫자 -> 텍스트)
decoded_sample = tokenizer.decode(
    tokenized_text[0][:20],  # 처음 20개 토큰
    skip_special_tokens=True  # 특수 토큰 제거
)

# 🔴 [추가 코드] 번역 시퀀스 생성 (Inference)
summary_ids = model.generate(
    tokenized_text,  # 인코더 입력 (토큰화된 영어 텍스트)
    max_length=150,  # 생성할 최대 길이
    num_beams=4,     # 빔 서치 크기 (4가 일반적)
    early_stopping=True
)

# 🔴 [추가 코드] 결과 디코딩 (숫자 ID -> 독일어 텍스트)
translated_text = tokenizer.decode(
    summary_ids.squeeze(),
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True
)

print("-" * 30)
print(f"생성된 번역문 (독일어 예상): {translated_text}")


------------------------------
생성된 번역문 (독일어 예상): Sie heiratete 10 Mal, manchmal nur innerhalb von zwei Wochen nacheinander.


<span style="color: pink"> 2. Q&A 기능
 

In [33]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

MODEL_NAME = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
# gpu 이동(가능하면)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

# 🔴 [질문 정의] - 이 기사에서 답을 찾을 질문
QUESTION = "How many times did Liana Barrientos get married in total?" 

# 전처리
# 1. 공백제거, 2. 줄바꿈을 공백으로 변환
preprecess_text = ARTICLE.strip().replace('\n'," ")

# 3. Task Prefix 추가: "question: [질문] context: [문맥]" QA 형식
input_text = f"question: {QUESTION} context: {preprecess_text}" 

# 토크나이제이션(인코딩)
tokenized_text = tokenizer.encode(
    input_text,return_tensors = 'pt',
    truncation = True, max_length = 512).to(device)

# --- 검증 코드 (생략 가능) ---
decoded_input_prefix = tokenizer.decode(
    tokenized_text[0][:10], skip_special_tokens=True 
)
print("-" * 30)
print(f"입력 검증: {decoded_input_prefix}")
print("-" * 30)
# -----------------------------

# 🔴 정답 시퀀스 생성 (Inference)
generated_ids = model.generate(
    tokenized_text,  
    max_length=20,   # 답변은 짧을 것이므로 20으로 설정
    num_beams=4,     
    early_stopping=True
)

# 🔴 결과 디코딩 (숫자 ID -> 정답 텍스트)
generated_answer = tokenizer.decode(
    generated_ids.squeeze(),
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True
)

print(f"**질문:** {QUESTION}")
print(f"**모델의 답변:** {generated_answer}")

------------------------------
입력 검증: question: How many times did Liana Barrien
------------------------------
**질문:** How many times did Liana Barrientos get married in total?
**모델의 답변:** 10
